In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

import os
os.chdir('/home/cat/code/manifolds/')

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange
from longitudinal_registration import longitudinal
from animal_database import animal_database

from scipy.io import loadmat
import matplotlib.patches as mpatches

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.spatial import ConvexHull, convex_hull_plot_2d

from matplotlib.path import Path
from tqdm import tqdm, trange

#
from __future__ import print_function
import cv2
import numpy as np

# 
np.set_printoptions(suppress=True)

Autosaving every 180 seconds


/home/cat/code/manifolds/animal_database/animal_database.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# NEW WORKBOOK STARTS HERE

In [3]:
#####################################################################
########## LOAD RAW DATA AND CONTOURS FROM CALCIUM FILES ############
#####################################################################
#
root_dir = '/media/cat/4TB/donato/'

animal_id = 'DON-008105'
ad = animal_database.AnimalDatabase()
# ad.load_sessions(animal_id)
# for k in range(len(ad.sessions)):
#     print (k,ad.sessions[k])
ad.sessions = ['20220118-1', '20220118-2']
    
#
l = longitudinal.Longitudinal(root_dir, animal_id, ad.sessions)
l.animal_id = animal_id
l.root_dir = root_dir

 
# load the multi session projetive transform
fname = '/media/cat/4TB/donato/DON-003343/ROIMatchPub_all_sessions.mat'
fname = '/media/cat/4TB/donato/DON-008105/match_file.mat'
l.make_homography_matrix_from_ROIMatchPub_clicks_load_multi_session(fname)

# get cell footprints for downstraema analysis
l.cell_boundary = 'convex_hull'  # 'convex_hull', 'concave_hull'
l.transform_cells_all_sessions()

#

 # of sessions to be transformed:  (2,)   (first session will have identity matrix)


In [4]:
###############################################
########## COMPUTE ALL PAIRS MATCHES ##########
###############################################
#
for sess1 in range(len(ad.sessions)-1):
    for sess2 in range(sess1+1, len(ad.sessions),1):
        print (sess1, sess2)
        l.parallel=True
        l.n_cores = 8
        l.recompute = False
        l.get_match_array_pairs(sess1, sess2)


0 1


In [5]:
# ######################################
# #### PLOT CONTOURS OF PAIR SESSION ###
# ######################################
# # set overlap threshold and generate plotting data
# l.thresh = 0.6
# sess1 = 0  
# sess2 = 6
# l.get_match_array_pairs(sess1, sess2)

# l.make_plotting_data_pair(sess1, sess2, l.match_array)  

#  # 
# #l.plot_overlap_masks()

# # # 
# l.plot_overlap_contours_pairs(sess1, sess2)

In [6]:
# ###########################################
# ####### PLOT PAIR WISE MATCH MATRIX #######
# ###########################################
# sessions = np.arange(10)
# n_cells = l.plot_match_matrix_multi(sessions)

# #
# plt.figure()
# plt.plot(sessions, n_cells)
# plt.xticks(sessions, l.sessions, rotation=45,fontsize=10)
# plt.ylim(bottom=0)
# plt.title("# of cells per session - DON-003343")
# plt.show()


In [7]:
###########################################
####### PLOT PAIR WISE MATCH MATRIX #######
###########################################
sessions = np.arange(10)
#sessions = [0,5,9]
#sessions = np.arange(0,10,2)
sessions = np.delete(np.arange(10),5)
sessions = [0,1]
#
l.thresh=0.6
l.root_dir = '/media/cat/4TB/donato/DON-008105/'
#res = l.get_match_array_list(sessions)
res = l.get_match_array_list2(sessions)

# 
cmap_name = 'Reds'
cmap = plt.get_cmap(cmap_name, len(sessions))

#
plt.figure()
ax=plt.subplot(111)
ax.set_facecolor("black")
for k in range(res.shape[0]):
    temp = res[k]
    for p in range(temp.shape[0]):
        c1 = temp[p]  # cell id
        temp2 = l.contours[sessions[p]][int(c1)]
        
        if k==0:
            plt.plot(temp2[:,0],temp2[:,1],
                    c=cmap(p), label=l.sessions[p])
        else:
            plt.plot(temp2[:,0],temp2[:,1],
                    c=cmap(p))
      
        
plt.legend(title="Sessions")
print (np.array(l.sessions)[sessions])        
plt.title("# cells: "+str(res.shape[0])+
         ", threshold:"+str(l.thresh))
plt.ylim(512,0)
plt.xlim(0,512)
plt.show()


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 33.39it/s]


all links:  (1065, 2)
[[   0.   nan]
 [   1.   47.]
 [   2.   nan]
 ...
 [1062.   nan]
 [1063.   nan]
 [1064.   nan]]
total # celsl found:  260
['20220118-1' '20220118-2']


In [35]:
#####################################################################
#
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-008105'
sessions = ['20220118-1', '20220118-2']
clrs=['black','red']
cmaps = ["Blues", "Reds"]
plt.figure(figsize=(10,10))

for ctr, session in enumerate(sessions):
    c = calcium.Calcium()
    c.root_dir = root_dir
    c.animal_id = animal_id
    c.session = session
    c.load_suite2p()
    c.cell_boundary = 'concave_hull'
    
    cell_ids = np.arange(c.F.shape[0])
    cmap = plt.get_cmap(cmaps[ctr], cell_ids.shape[0])
    idx = np.random.choice(np.arange(cell_ids.shape[0]), cell_ids.shape[0], replace=False)
    for cell in tqdm(cell_ids):
        contour = c.get_footprint_contour(cell,c.cell_boundary)
        plt.plot(contour[:,0],contour[:,1],
                c=cmap(idx[cell]))
        
plt.xlim(0,512)
plt.ylim(512,0)
plt.show()

100%|██████████████████████████████████████| 1065/1065 [00:02<00:00, 517.22it/s]


100%|████████████████████████████████████████| 616/616 [00:01<00:00, 595.16it/s]


In [ ]:
#
l = longitudinal.Longitudinal(root_dir, animal_id, sessions)
l.animal_id = animal_id
l.root_dir = root_dir

 
# load the multi session projetive transform
#fname = '/media/cat/4TB/donato/DON-003343/ROIMatchPub_all_sessions.mat'
#l.make_homography_matrix_from_ROIMatchPub_clicks_load_multi_session(fname)
l.hh_array = np.diag([1,1,1])
print (l.hh_array)


# get cell footprints for downstraema analysis
l.cell_boundary = 'convex_hull'  # 'convex_hull', 'concave_hull'
l.transform_cells_all_sessions()

# 

In [8]:
######
from PIL import Image
import numpy as np

def read_tiff(path):
    """
    path - Path to the multipage-tiff file
    """
    img = Image.open(path)
    images = []
    for i in range(img.n_frames):
        img.seek(i)
        images.append(np.array(img))
    return np.array(images)


sess1 = read_tiff("/media/cat/4TB/donato/DON-008105/20220118-1/002P-F/tif1/stack1.tif")
sess2 = read_tiff("/media/cat/4TB/donato/DON-008105/20220118-1/002P-F/tif2/stack2.tif")



In [31]:
##############################################################
##############################################################
##############################################################
# print (sess1.shape)
# print (sess2.shape)
# print (res)
size=512
x, y = np.meshgrid(np.arange(size),
                   np.arange(size))  # make a canvas with coordinates
x, y = x.flatten(), y.flatten()
points_mesh = np.vstack((x, y)).T

# 
session = '20220118-1'
c1 = calcium.Calcium()
c1.root_dir = root_dir
c1.animal_id = animal_id
c1.session = session
c1.load_suite2p()

# 
fig=plt.figure()
ctr=0
for k in range(500):
    ax=plt.subplot(1,3,ctr//50+1)
    if k in res[:,0]:
        continue
       
    # get missing cell footprint in session 1
    cell_id = k
    contour = c1.get_footprint_contour(cell_id,'concave_hull')
    mask = np.zeros((512,512))
    p = Path(contour)  # make a polygon
    grid = p.contains_points(points_mesh)
    footprint = grid.reshape(512,512)
    
    # get calcium activity in the missing ROI
    DFF = sess2[:,footprint].sum(axis=1)
    c1.high_cutoff = 14.9
    DFF = DFF-np.mean(DFF)
    DFF = c1.low_pass_filter(DFF.reshape(1,-1)).squeeze()
    DFF = DFF/50000
    plt.plot(DFF+ctr)
    ctr+=1
    
    if ctr>=150:
        break
    plt.ylabel("Neuron ID")
    
    plt.xlabel("Time (frame)")
plt.show()



low pass filter: 100%|███████████████████████████| 1/1 [00:00<00:00, 878.76it/s]


In [51]:
####################################################
########### COMPUTE MASKS FOR EACH CELL ############
####################################################
#
l.get_transformed_cell_masks()


processing session 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 480/480 [00:05<00:00, 91.34it/s]


processing session 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:04<00:00, 93.70it/s]


In [52]:
##########################################################
##### FIND INTERSECTION OF EACH CELL - POST ALIGNMENT ####
##########################################################
#
l.parallel=True
l.n_cores = 6
l.get_match_array()


  0%|          | 0/6 [00:00<?, ?it/s]

# res:  6
(6, 480, 393)


In [54]:
######################################
########## VISUALIZE RESULTS #########
######################################
# set overlap threshold and generate plotting data
l.thresh = 0.6
l.make_plotting_data()  

# 
l.plot_overlap_masks()

# 
l.plot_overlap_contours()

In [70]:

root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-006084'
session = '20210528'


c = calcium.Calcium()
c.root_dir = root_dir
c.animal_id = animal_id
c.session = session
c.load_suite2p()

c.load_footprints()



In [71]:
print (len(c.contours))
np.save('/home/cat/c28_contours.npy', c.contours, allow_pickle=True)

393


In [64]:
img_flattened = np.zeros((512*512,c.footprints.shape[0]))
img_sum = np.zeros((512,512))
for k in range(c.footprints.shape[0]):
    img_flattened[:,k] = c.footprints[k].flatten()
    img_sum+=c.footprints[k]
    
print(img_flattened.shape)    
print (img_sum.shape)

np.save('/home/cat/c28.npy', img_flattened)
np.save('/home/cat/c28_all.npy', img_sum)


(262144, 393)
(512, 512)


In [125]:
# ##################################################
# ##################################################
# ##################################################
# def find_homography(points1, points2, im):
    
#     # Find homography
#     h, mask = cv2.findHomography(points1, points2, 
#                                  cv2.RHO#cv2.RANSAC
#                                 )
#     print ("H: ", h)
#     # Use homography
#     height, width, channels = im.shape
#     im1Reg = cv2.warpPerspective(im, h, (width, height))

#     return im1Reg, h

# #
# # 
# im = np.zeros((imgs[0].shape[0],imgs[0].shape[0],3), dtype=np.uint8)
# im[:,:,0] = imgs[1]

# #
# pts_1 = pts_array[0][:5000]
# pts_2 = pts_array[1][:5000]
# imReg, h = find_homography(pts_1,
#                            pts_2, im)

# fig=plt.figure()
# # 
# ax=plt.subplot(1,2,1)
# plt.imshow(imgs[0]+imgs[1]*-1)
# plt.scatter(pts_array[0][:,1],pts_array[0][:,0], c = 'white')
# plt.scatter(pts_array[1][:,1],pts_array[1][:,0], c = 'red')

# # 
# ax=plt.subplot(1,2,2)
# #plt.imshow(imgs[0]+im2Reg[:,:,0]*-1)
# plt.imshow(imgs[0])


# plt.scatter(pts_array[0][:,1],pts_array[0][:,0], c='blue')

# from numpy.linalg import inv

# # invert transform matrix, created with cv2.perspectiveTransform
# M = inv(hh)  
# #M = inv(h)

# # 
# def warp_point(xy, M):
#     x= xy[0]
#     y=xy[1]
#     d = M[2, 0] * x + M[2, 1] * y + M[2, 2]

#     return (
#         int(M[0, 0] * x + M[0, 1] * y + M[0, 2] / d), # x
#         int(M[1, 0] * x + M[1, 1] * y + M[1, 2] / d), # y
#     )

# # 
# pts_trans = []
# for k in range(pts_array[1].shape[0]):
    
#     pts_trans.append(warp_point(pts_array[1][k], M))

# pts_trans = np.vstack(pts_trans)
# plt.scatter(pts_trans[:,1],
#             pts_trans[:,0], c='red')


# #
# idx = np.where(imgs[1]>0)
# idx22 = np.vstack(idx).T
# print (idx22.shape)

# #
# img_trans = []
# for k in range(idx22.shape[0]):
#     img_trans.append(warp_point(idx22[k], M))

# img_trans = np.vstack(img_trans)
# plt.scatter(img_trans[:,1],
#             img_trans[:,0],
#             s=10,
#             c='white')    

# # 
# plt.show()
# #plt.imshow(imReg)


error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'findHomography'
> Overload resolution failed:
>  - srcPoints is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'srcPoints'
>  - srcPoints is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'srcPoints'
